In [ ]:
# # Key principle: Assist, not diagnose or prescribe.

# Improves medication adherence
# Tracks symptoms safely
# Provides personalized support
# Escalates uncertainty to a human/doctor when needed
# Never replaces medical professionals

In [1]:
print("working")

working


In [2]:
# imports
from langgraph.graph import StateGraph , START , END
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage , BaseMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
import os

e:\X\projects\Medimimes\final main\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# class State(TypedDict):
#     user_id: int                             # Unique user identifier
#     user_input: str                          # Raw user message

#     # Safety Gate (Red Flag Node)
#     is_emergency: bool                       # True = hard stop
#     emergency_type: str | None               # Why flagged (chest pain, overdose, etc.)

#     # Intent Classifier Node (LLM)
#     intent: str | None                       # medication / symptom / adherence / education / motivation
#     route: str | None                        # Explicit next node name
#     query_type: str | None                  # Specific query type within intent for db query

#     # Domain Context
#     medication_context: dict                 # RAG output for drug info
#     adherence_history: dict                  # Past dose logs, patterns
#     symptom_logs: list                       # Structured symptoms list
#     behavioural_signal:str                   # User's motivation / barriers
#     adherence_summary: str                    # Summary of adherence history
#     education_content: str | None             # Educational text to share
#     db_query_result: dict | None        # Result from database query

#     # Final Output
#     response: str | None                     # Final text to return to user


In [ ]:

from typing import TypedDict, Optional, Dict, List, Any

class State(TypedDict, total=False):
    # ----------------------------------------------------
    # Backend Inputs
    # ----------------------------------------------------
    user_id: int                   # Authenticated Django user
    user_input: str                # Raw user message

    # ----------------------------------------------------
    # Safety Gate
    # ----------------------------------------------------
    is_emergency: bool             # True if red-flagged
    emergency_type: Optional[str]  # "chest_pain", "overdose", etc.

    # ----------------------------------------------------
    # Intent Classification
    # ----------------------------------------------------
    intent: Optional[str]          # medication / symptom / db_query / education / adherence / smalltalk
    query_type: Optional[str]      # Only for db_query intents

    # ----------------------------------------------------
    # Node Outputs (Structured Data)
    # ----------------------------------------------------
    medication_context: Optional[Dict[str, Any]]  # RAG output
    symptom_logs: Optional[List[Dict[str, Any]]]  # In-session only
    education_content: Optional[str]              # Education node output
    db_query_result: Optional[Dict[str, Any]]     # SQL results (structured)
    adherence_summary: Optional[Dict[str, Any]]   # For adherence conversation
    behavior_signal: Optional[str]                # Motivation tag ("mild", "struggle")
    adherence_history: dict                  # Past dose logs, patterns

    # ----------------------------------------------------
    # Final Output
    # ----------------------------------------------------
    response: Optional[str]         # Final message from response generator


In [5]:

# initiallize llm
load_dotenv()
llm=ChatGroq(model_name="llama-3.1-8b-instant",temperature=0.9)

In [6]:
# -----------red flag node--------------

def red_flag_node(State):
    text = State["user_input"].lower()

    #Helper to set emergency flags
    def flag(emergency_type):
        State["is_emergency"] = True
        State["emergency_type"] = emergency_type
        State["route"] = "emergency_node"
        return State

    # 1. Hard emergency signals
    EMERGENCY_SIGNS = {
        "chest pain": "cardiac_emergency",
        "severe chest pain": "cardiac_emergency",
        "difficulty breathing": "respiratory_distress",
        "can't breathe": "respiratory_distress",
        "shortness of breath": "respiratory_distress",
        "trouble breathing": "respiratory_distress",
        "fainted": "loss_of_consciousness",
        "unconscious": "loss_of_consciousness",
        "not waking up": "loss_of_consciousness",
    }

    for phrase, tag in EMERGENCY_SIGNS.items():
        if phrase in text:
            return flag(tag)

    # 2. Overdose indicators
    OVERDOSE_TERMS = [
        "took double dose",
        "took too many pills",
        "overdose",
        "accidentally took extra",
        "took two pills instead of one",
        "took more than prescribed",
    ]

    for term in OVERDOSE_TERMS:
        if term in text:
            return flag("possible_overdose")

    # 3. Severe allergic reactions
    ALLERGY_TERMS = [
        "swelling of face",
        "swelling of lips",
        "swelling of tongue",
        "swelling of throat",
        "can't swallow",
        "tightness in throat",
        "hives and trouble breathing",
        "skin turning blue",
    ]

    for term in ALLERGY_TERMS:
        if term in text:
            return flag("severe_allergic_reaction")

    # 4. Suicide / self harm
    SUICIDE_TERMS = [
        "i want to die",
        "i want to end my life",
        "i don't want to live",
        "i plan to hurt myself",
        "thinking to kill myself",
    ]

    for term in SUICIDE_TERMS:
        if term in text:
            return flag("mental_health_crisis")

    # 5. Severe acute symptoms
    ACUTE_TERMS = [
        "vomiting blood",
        "blood in vomit",
        "blood in stool",
        "vision loss",
        "seizure",
        "fit",
        "convulsion",
        "can't move arm",
        "can't move leg",
        "slurred speech",
    ]

    for term in ACUTE_TERMS:
        if term in text:
            return flag("acute_critical_symptom")

   
    State["is_emergency"] = False
    State["emergency_type"] = None
    State["route"] = "intent_classifier_node"
    return State


In [7]:
# ----------------emrgency node----------------------

def emergency_node(State: State):

    emergency_type = State.get("emergency_type")

    # Default fallback (should rarely trigger)
    message = (
        "This may be a medical emergency. Please seek immediate medical help or "
        "contact your local emergency services right now."
    )

    # -----------------------------
    # Custom messages by emergency type
    # -----------------------------
    TEMPLATES = {
        "cardiac_emergency": (
            "Chest pain or severe chest discomfort can be serious. "
            "Please seek emergency medical help immediately or call your local "
            "emergency services."
        ),

        "respiratory_distress": (
            "Difficulty breathing can be life-threatening. "
            "Please seek immediate medical help or call emergency services now."
        ),

        "possible_overdose": (
            "Taking more medication than prescribed can be dangerous. "
            "Please contact emergency medical services or go to the nearest hospital immediately."
        ),

        "severe_allergic_reaction": (
            "Signs of a severe allergic reaction can be life-threatening. "
            "Please seek urgent medical attention or call emergency services."
        ),

        "mental_health_crisis": (
            "I'm really sorry that you're feeling this way. You deserve immediate support. "
            "Please reach out to a trusted person, and contact emergency services or a local suicide prevention helpline right now."
        ),

        "acute_critical_symptom": (
            "The symptoms you're describing could be serious. "
            "Please seek urgent medical attention or contact emergency services immediately."
        )
    }

    # If a known emergency_type exists, use its message
    if emergency_type in TEMPLATES:
        message = TEMPLATES[emergency_type]

    # -----------------------------
    # Return safe template response
    # -----------------------------
    State["response"]=message
    State["route"]=None
    return State
    


In [8]:
INTENT_CLASSIFIER_PROMPT = """
You are an intent classification module for a medication adherence assistant.
Your job is ONLY to classify the user's message into a safe, predefined intent.
Do NOT generate answers or explanations.

Return a JSON dictionary ONLY, with these keys:

{
  "intent": "<one of: symptom_logging | medication_query | adherence_conversation | db_query | health_education | smalltalk | fallback>",
  "query_type": "<string or null>"
}

INTENT RULES:
-------------------------------------
1. symptom_logging  
   - User reports symptoms, feelings, side effects, discomfort
   - "I feel dizzy", "I have nausea", "I have a headache", etc.
   - NEVER diagnose, only classify.

2. medication_query  
   - Questions about what a medication does, how it works, why it's prescribed
   - "What is metformin?", "Why do people take BP medicine?", etc.

3. adherence_conversation  
   - User talks about missing/taking doses WITHOUT asking for history
   - "I missed my dose", "I already took it", "I forgot again", etc.

4. db_query  
   - User asks about their OWN medication history or schedule
   - Examples:
     - "Did I take my medicine yesterday?"
     - "When was my last missed dose?"
     - "How many doses did I miss this week?"
     - "What do I take today?"
   - For db_query, ALWAYS set query_type appropriately:
     Allowed query_type values:
       "last_missed_dose"
       "today_schedule"
       "weekly_summary"
       "dose_taken_yesterday"
       "recent_history"
       "upcoming_doses"
       "current_medications"

5. health_education
   - General health knowledge questions not about specific medication instructions
   - "What is hypertension?"
   - "Why is sleep important?"
   - "What happens when BP stays high?"

6. smalltalk
   - Greetings, compliments, jokes, general chat unrelated to health

7. fallback
   - If the intent is unclear or not allowed

CONSTRAINTS:
-------------------------------------
- NEVER diagnose.
- NEVER guess outside the allowed intents.
- NEVER recommend actions.
- If unsure, choose fallback.
- query_type must be null except when intent == db_query.

Now classify the user's message.
User: \"\"\"{user_input}\"\"\"  
Return ONLY the JSON.
"""
import json

def intent_classifier_node(state, llm):
    """
    LLM-based semantic intent classifier.
    Returns intent + optional query_type.
    """

    user_input = state["user_input"]

    prompt = INTENT_CLASSIFIER_PROMPT.format(user_input=user_input)

    # Call the LLM
    raw = llm.invoke(prompt)

    # Parse JSON safely
    try:
        if isinstance(raw, str):
            data = json.loads(raw)
        elif isinstance(raw, dict):
            data = raw
        else:
            data = json.loads(str(raw))
    except Exception:
        # Hard fallback
        data = {
            "intent": "fallback",
            "query_type": None
        }

    # Sanity checks
    allowed_intents = {
        "symptom_logging", "medication_query", "adherence_conversation",
        "db_query", "health_education", "smalltalk", "fallback"
    }

    if data.get("intent") not in allowed_intents:
        data["intent"] = "fallback"
        data["query_type"] = None

    # Route logic
    intent = data["intent"]

    route_map = {
        "symptom_logging": "symptom_node",
        "medication_query": "medication_rag_node",
        "adherence_conversation": "adherence_node",
        "db_query": "db_query_node",
        "health_education": "education_node",
        "smalltalk": "response_generation_node",
        "fallback": "response_generation_node"
    }

    
    state["intent"]= intent,
    state["query_type"]= data.get("query_type"),
    state[ "route"]=route_map[intent]
    
    return state



In [9]:
# --------------------------rag node-----------------------------
# 1. Load Structured CSV and Build Mini-Docs
import csv
from langchain.schema import Document

def load_medication_docs(csv_path):
    docs = []

    with open(csv_path, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            drug = row["drug"]

            def add_doc(field_name, field_value):
                if field_value and field_value.strip():
                    docs.append(Document(
                        page_content=f"{drug} — {field_name}: {field_value}",
                        metadata={"drug": drug, "field": field_name}
                    ))

            add_doc("Brands in India", row["brands_india"])
            add_doc("Indications", row["indications"])
            add_doc("Administration", row["administration"])
            add_doc("Notes", row["notes"])

    return docs

# embedding model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# vector store creation
import os
from langchain_community.vectorstores import Chroma

DB_PATH = "./med_db"

def create_vector_db(docs):
    return Chroma.from_documents(
        documents=docs,
        embedding=embedding_model,
        persist_directory=DB_PATH
    )

def load_vector_db():
    if os.path.exists(DB_PATH):
        return Chroma(
            persist_directory=DB_PATH,
            embedding_function=embedding_model
        )
    return None


# build vector db 
docs = load_medication_docs("resources/medicines_csv.csv")
vector_store = load_vector_db()

if vector_store is None:
    print("Building fresh medication DB...")
    vector_store = create_vector_db(docs)
else:
    print("Loaded existing DB.")

# retriever
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "lambda_mult": 0.7},
)


# medical_rag_node : just retrieves context from vector db
def medical_rag_node(State: State) -> State:
    query = State.get("user_input", "")
    MAX_TOKENS = 250

    def trim(x):
        return x[:MAX_TOKENS]

    try:
        docs = retriever.get_relevant_documents(query)
    except Exception as e:
        State["medical_rag_context"] = {
            "error": str(e),
            "results": [],
            "metadata": [],
            "empty": True
        }
        return State

    # If nothing found
    if not docs:
        State["medical_rag_context"] = {
            "results": [],
            "metadata": [],
            "empty": True
        }
        return State

    # Group by drug
    grouped = {}
    for d in docs:
        drug = d.metadata["drug"]
        grouped.setdefault(drug, []).append(trim(d.page_content))

    State["medical_rag_context"] = {
        "type": "medical_rag_context",
        "results": [trim(d.page_content) for d in docs],
        "metadata": [d.metadata for d in docs],
        "grouped_by_drug": grouped,
        "empty": False
    }

    return State




Building fresh medication DB...


In [16]:
# right now i am logging symptoms for in-session context only , later when appointment side will be made , and trend analysis component will be made in dashboard , i'll make neccesary changes

# ---------------Symptom logging node--------------
from chatbot.state import State
from chatbot.llm import llm
from datetime import datetime
import json

SYMPTOM_EXTRACTION_PROMPT = """
Extract structured symptom data from the user's message.

Return ONLY a JSON object with this exact format:

{
  "symptom": <string>,
  "severity": <string or null>,
  "duration": <string or null>
}

Rules:
- Do NOT diagnose.
- Do NOT infer unmentioned symptoms.
- Only extract what is explicitly stated.
- Severity can be: mild, moderate, severe, or null.
- Duration can be a phrase like "2 hours", "since morning", or null.
- If you are unsure, put null.
"""

def symptom_logging_node(state: State):

    user_text = state["user_input"]

    # 1. Run LLM extraction
    raw = llm.invoke(
        SYMPTOM_EXTRACTION_PROMPT + "\nUser message: " + user_text
    )

    # Normalize model output into Python dict
    try:
        if isinstance(raw, dict):
            parsed = raw
        else:
            parsed = json.loads(raw)
    except:
        parsed = {"symptom": None, "severity": None, "duration": None}

    # Extract values safely
    symptom = parsed.get("symptom")
    severity = parsed.get("severity")
    duration = parsed.get("duration")

    # 2. Build structured entry
    entry = {
        "symptom": symptom,
        "severity": severity,
        "duration": duration,
        "timestamp": datetime.utcnow().isoformat()
    }

    # 3. Append to symptom logs
    logs = state.get("symptom_logs", [])
    logs.append(entry)
    state["symptom_logs"] = logs

    # 4. DO NOT generate response text here
    # FinalResponseGeneratorNode will handle user messaging.

    return state


ModuleNotFoundError: No module named 'chatbot'

In [ ]:
# include the extracted in doctor appointmwent side later

# --------------------------Adherence Tracking Node --------------------------
# “I missed my dose today.”
# “I forgot last night’s medicine again.”
# “Did I take my pill today?”
# “I keep missing doses.”

from chatbot.state import State
from chatbot.llm import llm
from datetime import datetime
import json

ADHERENCE_EXTRACTION_PROMPT = """
Extract structured medication adherence information from the user's message.

Return ONLY a JSON object with this exact format:

{
  "event": "missed_dose" | "taken_dose" | "unsure",
  "dose_time": "morning" | "evening" | "night" | null,
  "date": "today" | "yesterday" | null,
  "reason": "<string or null>"
}

Rules:
- DO NOT diagnose.
- DO NOT give medical advice.
- Do not infer causes or consequences.
- Only extract what the user explicitly stated.
- If uncertain, return "unsure".
"""

def adherence_tracking_node(state: State):

    user_text = state["user_input"]

    # -----------------------------
    # 1. Run extraction using LLM
    # -----------------------------
    raw = llm.invoke(
        ADHERENCE_EXTRACTION_PROMPT + "\nUser message: " + user_text
    )

    # Normalize
    try:
        parsed = raw if isinstance(raw, dict) else json.loads(raw)
    except:
        parsed = {
            "event": "unsure",
            "dose_time": None,
            "date": None,
            "reason": None
        }

    event = parsed.get("event", "unsure")
    dose_time = parsed.get("dose_time")
    date = parsed.get("date")
    reason = parsed.get("reason")

    # -----------------------------
    # 2. Create adherence summary object
    # -----------------------------
    adherence_summary = {
        "event": event,
        "dose_time": dose_time,
        "date": date,
        "reason": reason,
        "timestamp": datetime.utcnow().isoformat()
    }

    # -----------------------------
    # 3. Behavior signal for motivation
    # -----------------------------
    if event == "missed_dose":
        behavior_signal = f"missed_{dose_time or 'unknown'}"
    elif event == "taken_dose":
        behavior_signal = f"taken_{dose_time or 'unknown'}"
    else:
        behavior_signal = "adherence_unsure"

    # -----------------------------
    # 4. Write to state (only allowed fields)
    # -----------------------------
    state["adherence_summary"] = adherence_summary
    state["behavior_signal"] = behavior_signal

    # NO route modification
    # NO response generation
    return state


In [12]:
# --------------Education node------------
HEALTH_EDUCATION_PROMPT = """
You are a neutral, factual health education assistant.

Your job:
- Explain general health, conditions, medications, or physiology in simple terms.
- Speak in GENERAL terms, not about the specific user.
- Do NOT diagnose.
- Do NOT give treatment recommendations.
- Do NOT suggest dose changes or specific drugs.
- Do NOT tell the user what they personally should do.
- Avoid fear-inducing language or worst-case dramatization.
- You may explain potential risks in a calm, high-level way, without saying they WILL happen.

Always:
- Use "in general", "people", "many patients", etc., instead of "you".
- Keep the explanation concise and focused.
- At the end, add: 
  "For personal medical advice or decisions, it's important to consult a healthcare professional."

Now, based on the user's question, provide a clear, short educational explanation.
"""


# ---------------------------------------------------------
# HEALTH EDUCATION NODE
# ---------------------------------------------------------

def health_education_node(State: State, llm):
    """
    Health Education Node:
    - Takes a general health/education question (already classified by intent node).
    - Uses LLM to generate a safe, non-diagnostic educational explanation.
    - Stores explanation in `education_content` for the final response generation node.
    - Does NOT directly produce the final chat response.
    """

    user_text = State["user_input"]

    # 1. Build prompt for LLM
    prompt = HEALTH_EDUCATION_PROMPT + "\n\nUser question:\n" + user_text

    # 2. Call LLM 
    try:
        explanation = llm.invoke(prompt)
        if isinstance(explanation, dict):
            # If your LLM wrapper returns structured output, adapt this as needed.
            explanation_text = explanation.get("text", "") or str(explanation)
        else:
            explanation_text = str(explanation)
    except Exception:
        # Fallback message if LLM call fails
        explanation_text = (
            "I’m unable to provide detailed information right now. "
            "For questions about health and medications, it's best to consult a healthcare professional."
        )

    # 3. Return partial state update
    # Final Response Generator Node will combine this with other context and send to user.

    State["education_content"]= explanation_text
    State[ "route"]= "response_generation_node"
    
    return State
    

In [13]:
# ------------------db_Query Node--------------------------
from datetime import datetime, timedelta

def db_query_node(state: dict, conn):
    """
    Safe, read-only DB query node for medication adherence.
    Works with your actual SQLite schema.
    
    INPUTS:
      state['query_type']  -> determined by Intent Classifier
      state['user_id']     -> authenticated user
    
    OUTPUTS:
      {
        "db_query_result": {...},  # structured data
        "route": "response_generation_node"
      }
    """

    query_type = state.get("query_type")
    user_id = state.get("user_id")

    cursor = conn.cursor()

    result_payload = {"query_type": query_type, "result": None}


    # =============================================
    # 1. LAST MISSED DOSE
    # =============================================
    if query_type == "last_missed_dose":

        cursor.execute("""
            SELECT 
                dl.id,
                m.pill_name,
                datetime(dl.scheduled_time),
                datetime(dl.timestamp),
                dl.status
            FROM medicines_doselog dl
            JOIN medicines_medication m
                ON dl.medication_id = m.id
            WHERE dl.user_id = ?
              AND dl.status = 'missed'
            ORDER BY dl.scheduled_time DESC
            LIMIT 1;
        """, (user_id,))

        row = cursor.fetchone()

        if row:
            result_payload["result"] = {
                "dose_id": row[0],
                "medication": row[1],
                "scheduled_time": row[2],
                "logged_time": row[3],
                "status": row[4]
            }


    # =============================================
    # 2. WEEKLY SUMMARY (taken/missed count)
    # =============================================
    elif query_type == "weekly_summary":

        seven_days_ago = (datetime.utcnow() - timedelta(days=7)).isoformat()

        cursor.execute("""
            SELECT status, COUNT(*)
            FROM medicines_doselog
            WHERE user_id = ?
              AND timestamp >= ?
            GROUP BY status;
        """, (user_id, seven_days_ago))

        rows = cursor.fetchall()

        summary = {"taken": 0, "missed": 0, "pending": 0}

        for status, count in rows:
            summary[status] = count

        result_payload["result"] = summary


    # =============================================
    # 3. DID I TAKE MY DOSE YESTERDAY?
    # =============================================
    elif query_type == "dose_taken_yesterday":

        yesterday = datetime.utcnow() - timedelta(days=1)

        start = yesterday.replace(hour=0, minute=0, second=0, microsecond=0).isoformat()
        end   = yesterday.replace(hour=23, minute=59, second=59, microsecond=0).isoformat()

        cursor.execute("""
            SELECT 
                m.pill_name,
                dl.status,
                datetime(dl.scheduled_time),
                datetime(dl.timestamp)
            FROM medicines_doselog dl
            JOIN medicines_medication m
                ON dl.medication_id = m.id
            WHERE dl.user_id = ?
              AND dl.scheduled_time BETWEEN ? AND ?
            ORDER BY dl.scheduled_time DESC
            LIMIT 1;
        """, (user_id, start, end))

        row = cursor.fetchone()

        if row:
            result_payload["result"] = {
                "medication": row[0],
                "status": row[1],
                "scheduled_time": row[2],
                "logged_time": row[3]
            }


    # =============================================
    # 4. TODAY'S DOSES
    # =============================================
    elif query_type == "today_schedule":

        cursor.execute("""
            SELECT 
                m.pill_name,
                m.dosage,
                m.times,
                dl.status,
                datetime(dl.scheduled_time),
                datetime(dl.timestamp)
            FROM medicines_medication m
            LEFT JOIN medicines_doselog dl
                ON dl.medication_id = m.id
                AND date(dl.scheduled_time) = date('now')
                AND dl.user_id = ?
            WHERE m.user_id = ?
            ORDER BY dl.scheduled_time ASC;
        """, (user_id, user_id))

        rows = cursor.fetchall()

        result_payload["result"] = [
            {
                "medication": r[0],
                "dosage": r[1],
                "scheduled_times": r[2],  # may be JSON or CSV
                "status": r[3],
                "scheduled_time": r[4],
                "logged_time": r[5]
            }
            for r in rows
        ]


    # =============================================
    # 5. UPCOMING DOSES (NEXT 5)
    # =============================================
    elif query_type == "upcoming_doses":

        cursor.execute("""
            SELECT 
                m.pill_name,
                m.dosage,
                datetime(dl.scheduled_time),
                dl.status
            FROM medicines_doselog dl
            JOIN medicines_medication m
                ON dl.medication_id = m.id
            WHERE dl.user_id = ?
              AND dl.scheduled_time > datetime('now')
            ORDER BY dl.scheduled_time ASC
            LIMIT 5;
        """, (user_id,))

        rows = cursor.fetchall()

        result_payload["result"] = [
            {
                "medication": r[0],
                "dosage": r[1],
                "scheduled_time": r[2],
                "status": r[3]
            }
            for r in rows
        ]


    # =============================================
    # 6. RECENT HISTORY (LAST 10 EVENTS)
    # =============================================
    elif query_type == "recent_history":

        cursor.execute("""
            SELECT 
                m.pill_name,
                datetime(dl.scheduled_time),
                datetime(dl.timestamp),
                dl.status
            FROM medicines_doselog dl
            JOIN medicines_medication m
                ON dl.medication_id = m.id
            WHERE dl.user_id = ?
            ORDER BY dl.scheduled_time DESC
            LIMIT 10;
        """, (user_id,))

        rows = cursor.fetchall()

        result_payload["result"] = [
            {
                "medication": r[0],
                "scheduled_time": r[1],
                "logged_time": r[2],
                "status": r[3]
            }
            for r in rows
        ]


    # =============================================
    # 7. DEFAULT — NO MATCH
    # =============================================
    else:
        result_payload["result"] = None


    cursor.close()

    
    state["db_query_result"]= result_payload
    state["route"] ="response_generation_node"
    


In [14]:
# -----------------------respobnse generation node-------------------------
def final_response_generator_node(state, llm):
    """
    Unified final message generator.
    All nodes pass structured information here.
    This node produces the ONLY user-facing message.
    
    Inputs it may receive:
      - state["education_content"]
      - state["db_query_result"]
      - state["medication_context"]
      - state["symptom_logs"]
      - state["adherence_summary"]
      - state["behavior_signal"]
      - state["intent"]
    
    Output:
      - {"response": "<final safe message>"}
    """

    intent = state.get("intent")
    education = state.get("education_content")
    db_result = state.get("db_query_result")
    med_info = state.get("medication_context")
    symptom_logs = state.get("symptom_logs", [])
    adherence_summary = state.get("adherence_summary")
    behavior_signal = state.get("behavior_signal")

    # ============================================================
    # BASE SAFETY + STYLE SYSTEM PROMPT
    # ============================================================

    SYSTEM_PROMPT = """
You are the final response generator for a safety-first medication adherence assistant.

Your rules:
- NEVER diagnose.
- NEVER recommend changing medication or dosage.
- NEVER give personalized medical advice.
- NEVER guess causes of symptoms.
- Keep tone calm, supportive, and non-judgmental.
- If discussing health risks, speak in general terms only.
- Encourage users to consult their healthcare provider for decisions.
- If user data was retrieved (DB results), present it factually without interpretation.
- Keep responses concise and clear.

Output ONLY the final chat response text.
"""

    # ============================================================
    # BUILD CONTEXT FOR LLM
    # ============================================================

    user_context = ""

    # 1. Education content
    if education:
        user_context += f"\n[EDUCATION_CONTENT]\n{education}\n"

    # 2. DB Query result
    if db_result:
        user_context += f"\n[DB_RESULT]\n{db_result}\n"

    # 3. Medication RAG Info
    if med_info:
        user_context += f"\n[MEDICATION_INFO]\n{med_info}\n"

    # 4. Symptom logs (session only)
    if symptom_logs:
        user_context += f"\n[SYMPTOM_LOGS]\n{symptom_logs}\n"

    # 5. Adherence conversation summary
    if adherence_summary:
        user_context += f"\n[ADHERENCE_BEHAVIOR]\n{adherence_summary}\n"

    # Behavior tag for motivational tone
    if behavior_signal:
        user_context += f"\n[BEHAVIOR_SIGNAL]\n{behavior_signal}\n"

    # ============================================================
    # ROUTING LOGIC (intents)
    # ============================================================

    # SIMPLE SMALLTALK
    if intent == "smalltalk":
        prompt = SYSTEM_PROMPT + f"""
The user is making smalltalk. Respond with a friendly, short message.

User message:
{state["user_input"]}
"""
        final_text = llm.invoke(prompt)
        return {"response": final_text}

    # FALLBACK
    if intent == "fallback":
        return {
            "response": (
                "I'm not completely sure how to help with that, but I can assist "
                "with medication information, symptom logging, reminders, or your dose history."
            )
        }

    # EDUCATION RESPONSE
    if intent == "health_education":
        prompt = SYSTEM_PROMPT + f"""
Use the [EDUCATION_CONTENT] to generate a final friendly, safe explanation.

{user_context}
"""
        final_text = llm.invoke(prompt)
        return {"response": final_text}

    # MEDICATION QUERY (RAG)
    if intent == "medication_query":
        prompt = SYSTEM_PROMPT + f"""
The user asked about medication information. Use [MEDICATION_INFO] safely.

{user_context}
"""
        final_text = llm.invoke(prompt)
        return {"response": final_text}

    # SYMPTOM LOGGING
    if intent == "symptom_logging":
        prompt = SYSTEM_PROMPT + f"""
Acknowledge the symptom, confirm it has been logged, and give non-medical safety advice.

{user_context}
"""
        final_text = llm.invoke(prompt)
        return {"response": final_text}

    # ADHERENCE CONVERSATION ("I missed my dose", etc.)
    if intent == "adherence_conversation":
        prompt = SYSTEM_PROMPT + f"""
The user described medication-taking behavior.
Use the behavior signal to give supportive, non-judgmental guidance.
Do NOT give medical consequences or advice.

{user_context}
"""
        final_text = llm.invoke(prompt)
        return {"response": final_text}

    # DB QUERY (dose history, schedule, etc.)
    if intent == "db_query":
        prompt = SYSTEM_PROMPT + f"""
Present the database results in a simple, factual, friendly way.
Do NOT interpret medically. Do NOT judge. Do NOT give advice.

{user_context}
"""
        final_text = llm.invoke(prompt)
        return {"response": final_text}

    # Should not reach here
    return {"response": "I'm here to help. Could you rephrase that?"}


In [15]:


def run_test_case(query):
    print(f"\n=== TEST QUERY: {query} ===")
    state = {"user_input": query}
    out = medical_rag_node(state)
    print(out)  # Pretty-print full output


if __name__ == "__main__":
    # Basic known-queries
    run_test_case("should i increase dosgae of parecetamol?")
    # run_test_case("Does paracetamol reduce fever?")
    run_test_case("When should I take pantoazole?")
    # run_test_case("What are common brands of azithromycin?")



=== TEST QUERY: should i increase dosgae of parecetamol? ===


C:\Users\naina\AppData\Local\Temp\ipykernel_16608\3362334901.py:80: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


{'user_input': 'should i increase dosgae of parecetamol?', 'medical_rag_context': {'type': 'medical_rag_context', 'results': ['Paracetamol — Notes: Avoid combining with other paracetamol-containing products.', 'Paracetamol — Indications: Relieves mild to moderate pain and reduces fever.', 'Ambroxol — Notes: Use as per leaflet.', 'ORS — Notes: Use WHO formula.'], 'metadata': [{'field': 'Notes', 'drug': 'Paracetamol'}, {'field': 'Indications', 'drug': 'Paracetamol'}, {'field': 'Notes', 'drug': 'Ambroxol'}, {'field': 'Notes', 'drug': 'ORS'}], 'grouped_by_drug': {'Paracetamol': ['Paracetamol — Notes: Avoid combining with other paracetamol-containing products.', 'Paracetamol — Indications: Relieves mild to moderate pain and reduces fever.'], 'Ambroxol': ['Ambroxol — Notes: Use as per leaflet.'], 'ORS': ['ORS — Notes: Use WHO formula.']}, 'empty': False}}

=== TEST QUERY: When should I take pantoazole? ===
{'user_input': 'When should I take pantoazole?', 'medical_rag_context': {'type': 'medi

In [ ]:
# build graph
graph=StateGraph(State)

In [ ]:
# add nodes

In [ ]:
# add edges

In [ ]:
# compile graph and make graph object

In [ ]:
# test queries